
# Minimum Enclosing Ball in 2D with CVXPY

We consider the *Minimum Enclosing Ball* problem. Given $m$ points $p_i \in \mathbb{R}^2$, the goal is to find the smallest Euclidean ball (center $c$, radius $r$) that contains all given points. 

\begin{aligned}
\text{minimize } \quad & r \\
\text{subject to }\quad & \|p_i - c\|_2 \le r,\quad i=1,\dots,m, \\
& r \ge 0, \\
& c \in \mathbb{R}^2, r \in \mathbb{R}. 
\end{aligned}

This is a convex optimization problem. 

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
%config InlineBackend.figure_formats = ['svg']

## Helper functions
We provide utilities to:
- print solver results,
- plot the point set together with the optimal enclosing ball.

In [ ]:
def print_results(problem, c_var = None, r_var = None):
    """Print status and optimal (c, r)."""
    print(f"Status: {problem.status}")
    if problem.value is not None:
        print(f"Optimal radius: {problem.value:.6g}")
    if c_var is not None and c_var.value is not None:
        cval = c_var.value
        print(f"c* = ({cval[0]:.6g}, {cval[1]:.6g})")
    if r_var is not None and r_var.value is not None:
        print(f"r* = {float(r_var.value):.6g}")


def plot_meb(P, c, r, title = None):
    """Plot 2D points and the minimum enclosing ball."""
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(P[:,0], P[:,1], c='tab:blue', s=40, label='Points')
    # Circle for the ball
    circ = Circle((c[0], c[1]), radius=r, facecolor='none', edgecolor='tab:red', lw=2, label='Minimum enclosing ball')
    ax.add_patch(circ)
    ax.scatter([c[0]], [c[1]], marker='*', s=200, c='tab:red', label='Center')
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, ls=':', alpha=0.5)
    ax.legend(loc='best')
    ax.set_title(title or 'Minimum Enclosing Ball')
    ax.set_xlabel('p1')
    ax.set_ylabel('p2')
    plt.show()


## CVXPY Formulation
We implement a function that solves the MEB:
- input: a 2D point cloud `P` of shape `(m,2)`;
- output: optimal center `c*`, radius `r*`, and the CVXPY `Problem` object.
The constraints are built with one `cp.norm` per point.


In [ ]:
# CVXPY formulation
def minimum_enclosing_ball(P):
    m = P.shape[0]

    c = cp.Variable(2, name='c')
    r = cp.Variable(name='r')

    constraints = [r >= 0]
    for i in range(m):
        constraints += [cp.norm(P[i] - c, 2) <= r]

    objective = cp.Minimize(r)
    prob = cp.Problem(objective, constraints)
    prob.solve()

    c_star = None if c.value is None else c.value.copy()
    r_star = None if r.value is None else float(r.value)
    return c_star, r_star, prob


## Example
We generate `40` points in 2D (two Gaussian clusters plus a few outliers), solve the MEB, and visualize the result.


In [ ]:
np.random.seed(1234)
# Two clusters
A = np.random.normal(loc=[-1.0, -0.5], scale=[0.3, 0.3], size=(18, 2))
B = np.random.normal(loc=[ 0.8,  0.9], scale=[0.35, 0.25], size=(18, 2))
# A few outliers
O = np.array([[1.8, -1.5], [-2.0, 1.6], [2.2, 2.0], [-2.2, -1.8]])
P = np.vstack([A, B, O])

# Solve MEB
c_star, r_star, prob = minimum_enclosing_ball(P)
print_results(prob, c_var=cp.Variable(2), r_var=cp.Variable())  # dummy call prints only status and problem.value
print(f"c* = ({c_star[0]:.6g}, {c_star[1]:.6g})")
print(f"r* = {r_star:.6g}")

# Verify max distance <= r*
max_dist = np.max(np.linalg.norm(P - c_star, axis=1))
print(f"Max distance to c*: {max_dist:.6g}")

# Plot
plot_meb(P, c_star, r_star, title='Minimum Enclosing Ball')